# Semantic Kernel と OpenBnB MCP サーバーの統合

このノートブックでは、Semantic Kernel を使用して、実際の OpenBnB MCP サーバーを利用し、MCPStdioPlugin を使って本物のAirbnb宿泊施設を検索する方法を示します。LLMアクセスには Azure AI Foundry を使用します。環境変数の設定については、[セットアップレッスン](/00-course-setup/README.md)を参照してください。


## 必要なパッケージをインポート


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCPプラグイン接続の作成

MCPStdioPluginを使用して[OpenBnB MCPサーバー](https://github.com/openbnb-org/mcp-server-airbnb)に接続します。このサーバーは、@openbnb/mcp-server-airbnbパッケージを通じてAirbnbの検索機能を提供します。


## クライアントの作成

このサンプルでは、Azure AI Foundryを使用してLLMにアクセスします。環境変数が正しく設定されていることを確認してください。


## 環境設定

Azure OpenAI の設定を構成します。以下の環境変数が設定されていることを確認してください:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP統合の理解

このノートブックは、**実際のOpenBnB MCPサーバー**に接続し、Airbnbの検索機能を提供します。

### 仕組み:

1. **MCPStdioPlugin**: MCPサーバーとの標準入力/出力通信を使用
2. **実際のNPMパッケージ**: `@openbnb/mcp-server-airbnb`をnpxでダウンロードして実行
3. **ライブデータ**: AirbnbのAPIから実際の物件データを返却
4. **機能の自動検出**: エージェントがMCPサーバーから利用可能な機能を自動的に検出

### 利用可能な機能:

OpenBnB MCPサーバーは通常、以下の機能を提供します:
- **search_listings** - 場所や条件でAirbnb物件を検索
- **get_listing_details** - 特定の物件の詳細情報を取得
- **check_availability** - 特定の日付の空室状況を確認
- **get_reviews** - 物件のレビューを取得
- **get_host_info** - 物件ホストの情報を取得

### 必要条件:

- **Node.js** がシステムにインストールされていること
- MCPサーバーパッケージをダウンロードするための**インターネット接続**
- **NPX** が利用可能であること（Node.jsに付属）

### 接続テスト:

MCPサーバーを手動でテストするには、以下を実行してください:
```bash
npx -y @openbnb/mcp-server-airbnb
```

これにより、OpenBnB MCPサーバーがダウンロードされ起動します。その後、Semantic Kernelが接続し、実際のAirbnbデータを取得します。


## OpenBnB MCPサーバーでエージェントを実行する

これから、OpenBnB MCPサーバーに接続して、ストックホルムで大人2人と子供1人向けの実際のAirbnb宿泊施設を検索するAIエージェントを実行します。検索条件を変更したい場合は、`user_inputs`リストを自由に編集してください。


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

概要  
おめでとうございます！Model Context Protocol (MCP) を使用して、実世界の宿泊施設検索と統合するAIエージェントの構築に成功しました。

使用した技術:  
- Semantic Kernel - Azure OpenAIを活用したインテリジェントエージェントの構築  
- Azure AI Foundry - LLM機能とチャット完了のため  
- MCP (Model Context Protocol) - 標準化されたツール統合のため  
- OpenBnB MCP Server - 実際のAirbnb検索機能のため  
- Node.js/NPX - 外部MCPサーバーの実行のため  

学んだこと:  
- MCP統合: Semantic Kernelエージェントを外部MCPサーバーに接続する方法  
- リアルタイムデータアクセス: 実際のAirbnb物件をライブAPIで検索する方法  
- プロトコル通信: エージェントとMCPサーバー間でstdio通信を使用する方法  
- 機能の発見: MCPサーバーから利用可能な機能を自動的に発見する方法  
- ストリーミングレスポンス: リアルタイムで関数呼び出しをキャプチャしログに記録する方法  
- HTMLレンダリング: スタイル付きテーブルやインタラクティブな表示でエージェントの応答をフォーマットする方法  

次のステップ:  
- 追加のMCPサーバーを統合する (天気、フライト、レストランなど)  
- MCPとA2Aプロトコルを組み合わせたマルチエージェントシステムを構築する  
- 独自のデータソース用にカスタムMCPサーバーを作成する  
- セッション間で永続的な会話メモリを実装する  
- MCPサーバーのオーケストレーションとともにエージェントをAzure Functionsにデプロイする  
- ユーザー認証と予約機能を追加する  

MCPアーキテクチャの主な利点:  
- 標準化: AIエージェントを外部ツールに接続するためのユニバーサルプロトコル  
- リアルタイムデータ: 様々なサービスから最新の情報にアクセス可能  
- 拡張性: 新しいデータソースやツールの簡単な統合  
- 相互運用性: 異なるAIフレームワークやエージェントプラットフォームで動作可能  
- 関心の分離: AIロジックと外部データアクセスを明確に分離  



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
